<a href="https://colab.research.google.com/github/Shashank-Karanam/AI-LaB/blob/main/Week9(FQL%20Resolution).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

name = "Shashank Ravindra Karanam"
usn  = "1BM23CS312"

print("Name:", name)
print("USN :", usn)
print("Topic: Prove 'John likes peanuts' using KB (forward chaining)")



# ---------------------------
# Part 2: Knowledge base + rules (forward chaining)
# We'll represent facts as tuples: ('predicate', arg1, arg2?, ...)
# e.g. ('food','apple'), ('eat','anil','peanuts'), ('alive','anil')
# ---------------------------

# Initial facts from the PPT
facts = set([
    ('food', 'apple'),
    ('food', 'vegetable'),
    ('eat', 'anil', 'peanuts'),
    ('alive', 'anil'),
    # the example says "Harry eats everything that Anil eats" -> handled as a rule
    # John likes all kinds of food -> handled as a rule
    # anything anyone eats and not killed is food -> rule
    # alive -> not killed and not killed -> alive (two-way) -> rules
])

# Helper to add a fact and know if it was new
def add_fact(f):
    if f not in facts:
        facts.add(f)
        # print the derivation as we add it
        print("Derived:", f)
        return True
    return False

# Rules (as functions that check facts and possibly add new ones)
def rule_alive_implies_not_killed():
    # alive(x) -> not_killed(x)
    added = False
    for (pred, *args) in list(facts):
        if pred == 'alive':
            x = args[0]
            added |= add_fact(('not_killed', x))
    return added

def rule_not_killed_implies_alive():
    # not_killed(x) -> alive(x)
    added = False
    for (pred, *args) in list(facts):
        if pred == 'not_killed':
            x = args[0]
            added |= add_fact(('alive', x))
    return added

def rule_eat_and_not_killed_implies_food():
    # eat(person, item) ∧ not_killed(person) -> food(item)
    added = False
    # collect persons with not_killed
    nk = {args[0] for (pred,*args) in facts if pred == 'not_killed'}
    for (pred, *args) in list(facts):
        if pred == 'eat':
            person, item = args
            if person in nk:
                added |= add_fact(('food', item))
    return added

def rule_harry_eats_everything_anil_eats():
    # eat(anil, X) -> eat(harry, X)
    added = False
    for (pred, *args) in list(facts):
        if pred == 'eat' and args[0] == 'anil':
            _, item = args
            added |= add_fact(('eat', 'harry', item))
    return added

def rule_john_likes_all_food():
    # food(x) -> likes(john, x)
    added = False
    for (pred, *args) in list(facts):
        if pred == 'food':
            item = args[0]
            added |= add_fact(('likes', 'john', item))
    return added

# Collect rule functions in a list for the forward-chaining loop
rules = [
    rule_alive_implies_not_killed,
    rule_not_killed_implies_alive,
    rule_eat_and_not_killed_implies_food,
    rule_harry_eats_everything_anil_eats,
    rule_john_likes_all_food
]

# Run forward chaining until no new facts appear
changed = True
iteration = 0
print("Initial facts:", facts, "\n")
while changed:
    iteration += 1
    changed = False
    # apply each rule once per iteration
    for r in rules:
        if r():
            changed = True
    # safety: break if too many iterations (shouldn't happen here)
    if iteration > 20:
        print("Stopping after 20 iterations (safety).")
        break

print("\nFinal facts:")
for f in sorted(facts):
    print(" ", f)

# ---------------------------
# Check the query: likes(john, peanuts)
# ---------------------------
query = ('likes', 'john', 'peanuts')
print("\nQuery:", query)
print("Proved:" , query in facts)   # will print True if derived


Name: Shashank Ravindra Karanam
USN : 1BM23CS312
Topic: Prove 'John likes peanuts' using KB (forward chaining)
Initial facts: {('food', 'vegetable'), ('eat', 'anil', 'peanuts'), ('food', 'apple'), ('alive', 'anil')} 

Derived: ('not_killed', 'anil')
Derived: ('food', 'peanuts')
Derived: ('eat', 'harry', 'peanuts')
Derived: ('likes', 'john', 'apple')
Derived: ('likes', 'john', 'peanuts')
Derived: ('likes', 'john', 'vegetable')

Final facts:
  ('alive', 'anil')
  ('eat', 'anil', 'peanuts')
  ('eat', 'harry', 'peanuts')
  ('food', 'apple')
  ('food', 'peanuts')
  ('food', 'vegetable')
  ('likes', 'john', 'apple')
  ('likes', 'john', 'peanuts')
  ('likes', 'john', 'vegetable')
  ('not_killed', 'anil')

Query: ('likes', 'john', 'peanuts')
Proved: True
